In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests, pacf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
warnings.filterwarnings("ignore")
import scipy
import datetime

from joblib import Parallel, delayed
import os
import statsmodels

In [ ]:
#director = './OLD_RESULT/'
director = './'
resultsfiles = [director+x for x in os.listdir(director) if 'result_' in x ]

loinc1 = '0000000'
#loinc1 = '1920-8'
loinc2 = '17861-6'
loinc3 = '2160-0'
loinc1 = loinc1.replace('-','_')
loinc2 = loinc2.replace('-','_')
loinc3 = loinc3.replace('-','_')
targetbool = (pd.DataFrame(resultsfiles)[0].str.contains(loinc1).astype(int) + pd.DataFrame(resultsfiles)[0].str.contains(loinc2).astype(int) + pd.DataFrame(resultsfiles)[0].str.contains(loinc3).astype(int))==2
resultsfiles = pd.DataFrame(resultsfiles)[targetbool].iloc[:,0].to_numpy()
resultsfiles

In [ ]:
csvs = list()
for file in resultsfiles:
    try:
        csvtemp = pd.read_csv(file).drop('Unnamed: 0',axis=1, errors='ignore').dropna()
        csvs.append(csvtemp)
    except:
        pass
#csvt = pd.concat(csvs)
#csvt = csvt[(csvt['Num_Diff1']!=19) & (csvt['Num_Diff2']!=19)]
def preprocess(csvt):
    pvalue1_adj = statsmodels.stats.multitest.multipletests(csvt['pvalue1'])
    pvalue2_adj = statsmodels.stats.multitest.multipletests(csvt['pvalue2'])

    csvt['pvalue1_adj'] = pvalue1_adj[1]
    csvt['pvalue2_adj'] = pvalue2_adj[1]

    csvt['1xmap2_adj'] = csvt['1xmap2'] & (csvt['pvalue1_adj']<0.05)
    csvt['2xmap1_adj'] = csvt['2xmap1'] & (csvt['pvalue2_adj']<0.05)
    return csvt

csvs = [preprocess(x) for x in csvs]

In [ ]:
df = csvs[0].copy()
df['1xmap2_only'] = df['1xmap2_adj'] & ~ df['2xmap1_adj']
df['2xmap1_only'] = df['2xmap1_adj'] & ~ df['1xmap2_adj']
df['bidiredction'] = df['2xmap1_adj'] & df['1xmap2_adj']

In [ ]:
str1 = df['feature1'].iloc[0]+' <-  '+df['feature2'].iloc[0]+ ' : '+str(df['1xmap2_only'].sum())
str2 = df['feature1'].iloc[0]+'  -> '+df['feature2'].iloc[0]+ ' : '+str(df['2xmap1_only'].sum())
str3 = df['feature1'].iloc[0]+' <-> '+df['feature2'].iloc[0]+ ' : '+str(df['bidiredction'].sum())

In [ ]:
print(df['site'].iloc[0])
print(str1)
print(str2)
print(str3)
print(df.shape[0])

In [ ]:
df.columns

In [ ]:
df[df['1xmap2_only']]['rho1_last'].mean()

In [ ]:
df[df['2xmap1_only']]['rho2_last'].mean()

In [ ]:
csvs[1][['PATID' ,'ENCOUNTERID', 'feature1', 'feature2', '1xmap2_adj','2xmap1_adj']].merge(csvs[2][['PATID' ,'ENCOUNTERID', 'feature1', 'feature2', '1xmap2_adj','2xmap1_adj']], on=['PATID' ,'ENCOUNTERID'], how='left')

In [ ]:
csvs[0][['PATID' ,'ENCOUNTERID', 'feature1', 'feature2', '1xmap2_adj','2xmap1_adj']].merge(csvs[2][['PATID' ,'ENCOUNTERID', 'feature1', 'feature2', '1xmap2_adj','2xmap1_adj']], on=['PATID' ,'ENCOUNTERID'], how='left')

In [ ]:
csvs[0][['PATID' ,'ENCOUNTERID', 'feature1', 'feature2', '1xmap2_adj','2xmap1_adj']].merge(csvs[1][['PATID' ,'ENCOUNTERID', 'feature1', 'feature2', '1xmap2_adj','2xmap1_adj']], on=['PATID' ,'ENCOUNTERID'], how='left')